# Poisson(5) with CS rho = 0.9, sigma2 = 1.0

For Poisson Base with $\lambda = 5$, with the correlation $\rho = 0.9, \sigma^2 = 1.0,$

The theoretical correlation is a function of $d_i, \rho, \sigma^2, kt$

$$Corr(Y_k, Y_l)
 = \frac{\rho * \sigma^2}{1 + \frac{1}{2} * \sigma^2 * kt + \frac{1}{2} * \sigma^2 (d_i - 1)}$$

We have $\lambda = 5,$ thus we have kurtosis $kt = kt(\lambda) = 3.2.$

Let's see what happens to the theoretical/empirical correlation when we fix $\rho = 0.9, \sigma^2 = 1.0$ and range cluster sizes $d_i$

$$Corr(Y_k, Y_l)
 = \frac{0.9}{1 + \frac{1}{2} * 3.2 + \frac{1}{2} * (d_i - 1)} =  \frac{0.9}{1 + \frac{1}{2} * (d_i + 2.2)} $$

# We will show:
    1) Simulating under QC model, the theoretical and empirical correlation IS a function of cluster sizes di

    2) Simulating under GLMM, the theoretical and empirical correlation is NOT function of cluster sizes di
    

## TOC:

# di = 2
* [CS di = 2](#ex0)
* [simulate under GLMM CS di = 2](#ex0g)

# di = 5
* [CS di = 5](#ex1)
* [simulate under GLMM CS di = 5](#ex1g)

# di = 10 
* [CS di = 10](#ex2)
* [simulate under GLMM CS di = 10](#ex2g)

# di = 25
* [CS di = 25](#ex3)
* [simulate under GLMM CS di = 25](#ex3g)

# Comparisons
* [Theoretical vs. Empirical Correlation Simulated under QC](#ex4)
* [Empirical Correlation Simulated under GLMM](#ex4g)

In [1]:
using GLMCopula, DelimitedFiles, LinearAlgebra, Random, GLM, MixedModels, CategoricalArrays
using Random, Roots, SpecialFunctions, StatsBase
using DataFrames, DelimitedFiles, Statistics, ToeplitzMatrices
import StatsBase: sem

In [2]:
function get_V_CS(ρ, n)
    vec = zeros(n)
    vec[1] = 1.0
    for i in 2:n
        vec[i] = ρ
    end
    V = ToeplitzMatrices.SymmetricToeplitz(vec)
    V
end

get_V_CS (generic function with 1 method)

In [3]:
# true parameter values
βtrue = [log(5.0)]
σ2true = [1.0]
ρtrue = [0.9]
lambda = [5]

samplesize = 100000 # number of sampling units

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64

Float64

# Kurtosis of each Poisson(5) base distribution is 3.2 

We have $\lambda = 5,$ thus we have kurtosis $kt = kt(\lambda) = 3.2.$

Let's see what happens when we fix $\rho = 0.9, \sigma^2 = 1.0, kt = 3.2$ and range cluster sizes $d_i$

In [4]:
d = Poisson(lambda[1])
μ, σ², sk, kt = mean(d), var(d), skewness(d), kurtosis(d, false)

(5.0, 5.0, 0.4472135954999579, 3.2)

## CS di = 2 <a class="anchor" id="ex0"></a>

$d_i = 2$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [5]:
di = 2 # number of observations per cluster
V_CS = get_V_CS(ρtrue[1], di)

# true Gamma
Γ_CS = σ2true[1] * V_CS

2×2 Matrix{Float64}:
 1.0  0.9
 0.9  1.0

In [6]:
vecd = Vector{DiscreteUnivariateDistribution}(undef, di)
for i in 1:di
    vecd[i] = Poisson(5.0)
end
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ_CS)

Random.seed!(12345)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, samplesize)

gcs = Vector{GLMCopulaCSObs{T, D, Link}}(undef, samplesize)
for i in 1:samplesize
    X = ones(di, 1)
    y = Float64.(Y_nsample[i])
    V = [Float64.(Matrix(I, di, di))]
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
di = length(gcm.data[1].y)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_2 = StatsBase.cor(Y_CS)

  1.155270 seconds (5.64 M allocations: 627.854 MiB, 8.34% gc time, 47.19% compilation time)


2×2 Matrix{Float64}:
 1.0       0.285195
 0.285195  1.0

In [7]:
theoretical_rho_2_kurtosis = (ρtrue[1] * σ2true[1]) / (1 + ((di/2) * σ2true[1]) + (0.5 * (kt - 1) * σ2true[1]))

0.2903225806451613

In [8]:
mean_empirical_cor_2 = mean(GLMCopula.offdiag(empirical_cor_2))

0.2851948656513909

## simulate under GLMM CS di = 2 <a class="anchor" id="ex0g"></a>

$d_i = 2$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [9]:
function __get_distribution(dist::Type{D}, μ) where D <: UnivariateDistribution
    return dist(μ)
end

for i in 1:samplesize
    X = ones(di, 1)
    η = X * βtrue
    # generate mvn response
    mvn_d = MvNormal(η, Γ_CS)
    mvn_η = rand(mvn_d)
    μ = GLM.linkinv.(link, mvn_η)
    y = Float64.(rand.(__get_distribution.(D, μ)))
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_2_GLMM = StatsBase.cor(Y_CS)

2×2 Matrix{Float64}:
 1.0       0.798455
 0.798455  1.0

In [10]:
mean_empirical_cor_2_GLMM = mean(GLMCopula.offdiag(empirical_cor_2_GLMM))

0.7984547527377245

## CS di = 5 <a class="anchor" id="ex1"></a>

$d_i = 5$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [11]:
di = 5 # number of observations per cluster
V_CS = get_V_CS(ρtrue[1], di)

# true Gamma
Γ_CS = σ2true[1] * V_CS

5×5 Matrix{Float64}:
 1.0  0.9  0.9  0.9  0.9
 0.9  1.0  0.9  0.9  0.9
 0.9  0.9  1.0  0.9  0.9
 0.9  0.9  0.9  1.0  0.9
 0.9  0.9  0.9  0.9  1.0

In [12]:
vecd = Vector{DiscreteUnivariateDistribution}(undef, di)
for i in 1:di
    vecd[i] = Poisson(5.0)
end
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ_CS)

Random.seed!(12345)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, samplesize)

gcs = Vector{GLMCopulaCSObs{T, D, Link}}(undef, samplesize)
for i in 1:samplesize
    X = ones(di, 1)
    y = Float64.(Y_nsample[i])
    V = [Float64.(Matrix(I, di, di))]
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
di = length(gcm.data[1].y)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_5 = StatsBase.cor(Y_CS)

  2.099954 seconds (9.00 M allocations: 1.299 GiB, 31.70% gc time)


5×5 Matrix{Float64}:
 1.0       0.196943  0.197756  0.195678  0.193317
 0.196943  1.0       0.194243  0.19766   0.192144
 0.197756  0.194243  1.0       0.190767  0.196116
 0.195678  0.19766   0.190767  1.0       0.195628
 0.193317  0.192144  0.196116  0.195628  1.0

In [13]:
theoretical_rho_5_kurtosis = (ρtrue[1] * σ2true[1]) / (1 + ((di/2) * σ2true[1]) + (0.5 * (kt - 1) * σ2true[1]))

0.1956521739130435

In [14]:
mean_empirical_cor_5 = mean(GLMCopula.offdiag(empirical_cor_5))

0.19502545069070262

## simulate under GLMM CS di = 5 <a class="anchor" id="ex1g"></a>

$d_i = 5$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [15]:
for i in 1:samplesize
    X = ones(di, 1)
    η = X * βtrue
    # generate mvn response
    mvn_d = MvNormal(η, Γ_CS)
    mvn_η = rand(mvn_d)
    μ = GLM.linkinv.(link, mvn_η)
    y = Float64.(rand.(__get_distribution.(D, μ)))
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_5_GLMM = StatsBase.cor(Y_CS)

5×5 Matrix{Float64}:
 1.0       0.792778  0.796232  0.797336  0.787295
 0.792778  1.0       0.787723  0.791998  0.788881
 0.796232  0.787723  1.0       0.796937  0.787417
 0.797336  0.791998  0.796937  1.0       0.782993
 0.787295  0.788881  0.787417  0.782993  1.0

In [16]:
mean_empirical_cor_5_GLMM = mean(GLMCopula.offdiag(empirical_cor_5_GLMM))

0.7909588752324979

## CS di = 10 <a class="anchor" id="ex2"></a>

$d_i = 10$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [17]:
di = 10 # number of observations per cluster
V_CS = get_V_CS(ρtrue[1], di)

# true Gamma
Γ_CS = σ2true[1] * V_CS

10×10 Matrix{Float64}:
 1.0  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  1.0  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  1.0  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  1.0  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  1.0  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  1.0  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  1.0  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  1.0  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  1.0  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  1.0

In [18]:
vecd = Vector{DiscreteUnivariateDistribution}(undef, di)
for i in 1:di
    vecd[i] = Poisson(5.0)
end
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ_CS)

Random.seed!(1234)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, samplesize)

gcs = Vector{GLMCopulaCSObs{T, D, Link}}(undef, samplesize)
for i in 1:samplesize
    X = ones(di, 1)
    y = Float64.(Y_nsample[i])
    V = [Float64.(Matrix(I, di, di))]
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
di = length(gcm.data[1].y)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_10 = StatsBase.cor(Y_CS)

  4.228739 seconds (17.50 M allocations: 3.017 GiB, 24.10% gc time)


10×10 Matrix{Float64}:
 1.0       0.128016  0.130614  0.129047  …  0.128288  0.125534  0.133939
 0.128016  1.0       0.120538  0.125782     0.127935  0.127707  0.130416
 0.130614  0.120538  1.0       0.125045     0.128896  0.126352  0.123502
 0.129047  0.125782  0.125045  1.0          0.122533  0.128499  0.129321
 0.123802  0.121636  0.125933  0.12542      0.123983  0.127865  0.129307
 0.126466  0.123785  0.127689  0.123933  …  0.122117  0.128876  0.125273
 0.124606  0.124493  0.129822  0.12864      0.12297   0.127572  0.127659
 0.128288  0.127935  0.128896  0.122533     1.0       0.122736  0.128858
 0.125534  0.127707  0.126352  0.128499     0.122736  1.0       0.12844
 0.133939  0.130416  0.123502  0.129321     0.128858  0.12844   1.0

In [19]:
theoretical_rho_10_kurtosis = (ρtrue[1] * σ2true[1]) / (1 + ((di/2) * σ2true[1]) + (0.5 * (kt - 1) * σ2true[1]))

0.1267605633802817

In [20]:
mean_empirical_cor_10 = mean(GLMCopula.offdiag(empirical_cor_10))

0.12640711338368954

## simulate under GLMM CS di = 10 <a class="anchor" id="ex2g"></a>

$d_i = 10$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [21]:
for i in 1:samplesize
    X = ones(di, 1)
    η = X * βtrue
    # generate mvn response
    mvn_d = MvNormal(η, Γ_CS)
    mvn_η = rand(mvn_d)
    μ = GLM.linkinv.(link, mvn_η)
    y = Float64.(rand.(__get_distribution.(D, μ)))
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_10_GLMM = StatsBase.cor(Y_CS)

10×10 Matrix{Float64}:
 1.0       0.795277  0.788873  0.797751  …  0.797274  0.791648  0.78627
 0.795277  1.0       0.790251  0.797387     0.794843  0.792892  0.787881
 0.788873  0.790251  1.0       0.788445     0.801963  0.794749  0.789376
 0.797751  0.797387  0.788445  1.0          0.800243  0.794788  0.793074
 0.79228   0.793276  0.794363  0.793684     0.79594   0.797735  0.792796
 0.79759   0.797007  0.790109  0.79892   …  0.799824  0.794066  0.788899
 0.792135  0.793612  0.783496  0.794305     0.790397  0.784503  0.788064
 0.797274  0.794843  0.801963  0.800243     1.0       0.797156  0.793496
 0.791648  0.792892  0.794749  0.794788     0.797156  1.0       0.788979
 0.78627   0.787881  0.789376  0.793074     0.793496  0.788979  1.0

In [22]:
mean_empirical_cor_10_GLMM = mean(GLMCopula.offdiag(empirical_cor_10_GLMM))

0.7931953599572975

## CS di = 25 <a class="anchor" id="ex3"></a>

$d_i = 25$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [23]:
di = 25 # number of observations per cluster
V_CS = get_V_CS(ρtrue[1], di)

# true Gamma
Γ_CS = σ2true[1] * V_CS

25×25 Matrix{Float64}:
 1.0  0.9  0.9  0.9  0.9  0.9  0.9  0.9  …  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  1.0  0.9  0.9  0.9  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  1.0  0.9  0.9  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  1.0  0.9  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  1.0  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  1.0  0.9  0.9  …  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  1.0  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  1.0     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  …  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9     0.9  0.9  0.9  0.9  0.9  0.9  0.9
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  

In [24]:
vecd = Vector{DiscreteUnivariateDistribution}(undef, di)
for i in 1:di
    vecd[i] = Poisson(5.0)
end
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ_CS)

Random.seed!(1234)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, samplesize)

gcs = Vector{GLMCopulaCSObs{T, D, Link}}(undef, samplesize)
for i in 1:samplesize
    X = ones(di, 1)
    y = Float64.(Y_nsample[i])
    V = [Float64.(Matrix(I, di, di))]
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
di = length(gcm.data[1].y)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_25 = StatsBase.cor(Y_CS)

 12.662374 seconds (43.00 M allocations: 14.642 GiB, 20.92% gc time)


25×25 Matrix{Float64}:
 1.0        0.0577683  0.0610863  …  0.0554469  0.0615061  0.0568422
 0.0577683  1.0        0.0594631     0.0591781  0.0682081  0.0606417
 0.0610863  0.0594631  1.0           0.0589024  0.0581182  0.0614743
 0.0605585  0.0658646  0.0611926     0.0641798  0.0639662  0.0669419
 0.0623432  0.0616572  0.0626698     0.0589285  0.0640179  0.0662437
 0.0636397  0.062638   0.0620784  …  0.0582243  0.0684209  0.0614459
 0.0623193  0.0625838  0.0600198     0.0610188  0.0588353  0.0650894
 0.0610067  0.0580055  0.0530576     0.0618378  0.0560169  0.0627076
 0.0575952  0.0655842  0.0618869     0.0573538  0.0638674  0.0608164
 0.0617213  0.0644334  0.0617497     0.0571646  0.0650496  0.0655976
 0.0617591  0.0635662  0.0651132  …  0.0615142  0.0607624  0.0617458
 0.0550982  0.0636181  0.0583746     0.0595954  0.0552984  0.0617601
 0.062011   0.064116   0.0638051     0.0607127  0.0550305  0.0633031
 0.0558988  0.0612153  0.0646206     0.0580648  0.0597249  0.0656671
 0.0602253 

In [25]:
theoretical_rho_25_kurtosis = (ρtrue[1] * σ2true[1]) / (1 + ((di/2) * σ2true[1]) + (0.5 * (kt - 1) * σ2true[1]))

0.06164383561643836

In [26]:
mean_empirical_cor_25 = mean(GLMCopula.offdiag(empirical_cor_25))

0.06127455295234851

## simulate under GLMM CS di = 25 <a class="anchor" id="ex3g"></a>

$d_i = 25$

$\lambda = 5, \rho = 0.9, \sigma^2 = 1.0$

In [27]:
for i in 1:samplesize
    X = ones(di, 1)
    η = X * βtrue
    # generate mvn response
    mvn_d = MvNormal(η, Γ_CS)
    mvn_η = rand(mvn_d)
    μ = GLM.linkinv.(link, mvn_η)
    y = Float64.(rand.(__get_distribution.(D, μ)))
    gcs[i] = GLMCopulaCSObs(y, X, d, link)
end

# form model
gcm = GLMCopulaCSModel(gcs);

N = length(gcm.data)
Y_CS = zeros(N, di)
for j in 1:di
    Y_CS[:, j] = [gcm.data[i].y[j] for i in 1:N]
end
empirical_cor_25_GLMM = StatsBase.cor(Y_CS)

25×25 Matrix{Float64}:
 1.0       0.794896  0.79396   0.791151  …  0.7956    0.791991  0.79801
 0.794896  1.0       0.795116  0.797399     0.803492  0.797091  0.790531
 0.79396   0.795116  1.0       0.793622     0.799789  0.795092  0.794984
 0.791151  0.797399  0.793622  1.0          0.798121  0.796452  0.794455
 0.79591   0.798614  0.79625   0.797645     0.801769  0.793705  0.792677
 0.796883  0.794082  0.79092   0.792924  …  0.797837  0.796791  0.79716
 0.791098  0.790016  0.790738  0.790949     0.795933  0.79284   0.791257
 0.793473  0.794692  0.796365  0.797031     0.793969  0.792996  0.794117
 0.792762  0.796588  0.795166  0.793261     0.793599  0.788243  0.789603
 0.791031  0.796464  0.793903  0.796603     0.795217  0.79615   0.795594
 0.799324  0.796172  0.790406  0.79471   …  0.794893  0.793752  0.795172
 0.791673  0.794959  0.793313  0.788509     0.795889  0.794092  0.791398
 0.796811  0.79305   0.794413  0.796343     0.800754  0.797063  0.79923
 0.790201  0.793111  0.790488  

In [28]:
mean_empirical_cor_25_GLMM = mean(GLMCopula.offdiag(empirical_cor_25_GLMM))

0.7939346055300571

## Comparisons 

##  1. Theoretical vs. Empirical Correlation simulated under QC <a class="anchor" id="ex4"></a>

### Takeaway: Quasi-Copula correlation IS a function of the di, kt, $\rho$ and $\sigma^2$

The theoretical correlation is a function of $d_i, \rho, \sigma^2, kt(\lambda)$

$$Corr(Y_k, Y_l)
 = \frac{\rho * \sigma^2}{1 + \frac{1}{2} * \sigma^2 * kt + \frac{1}{2} * \sigma^2 (d_i - 1)}$$

We have $\lambda = 5,$ thus we have kurtosis $kt = kt(\lambda) = 3.2.$

Let's see what happens to the theoretical/empirical correlation when we fix $\rho = 0.9, \sigma^2 = 1.0$ and range cluster sizes $d_i$

$$Corr(Y_k, Y_l)
 = \frac{0.9}{1 + \frac{1}{2} * 3.2 + \frac{1}{2} * (d_i - 1)} =  \frac{0.9}{1 + \frac{1}{2} * (d_i + 2.2)} $$


In [29]:
# di = 2
[theoretical_rho_2_kurtosis mean_empirical_cor_2]

1×2 Matrix{Float64}:
 0.290323  0.285195

In [30]:
# di = 5
[theoretical_rho_5_kurtosis mean_empirical_cor_5]

1×2 Matrix{Float64}:
 0.195652  0.195025

In [31]:
# di = 10
[theoretical_rho_10_kurtosis mean_empirical_cor_10]

1×2 Matrix{Float64}:
 0.126761  0.126407

In [32]:
# di = 25
[theoretical_rho_25_kurtosis mean_empirical_cor_25]

1×2 Matrix{Float64}:
 0.0616438  0.0612746

##  2. Empirical Correlation simulated under GLMM<a class="anchor" id="ex4g"></a>

### Takeaway: GLMM correlation is NOT a function of the cluster size (Only a function of $\rho$ and $\sigma^2)$

In [33]:
# glmm correlation di = 2 
mean_empirical_cor_2_GLMM

0.7984547527377245

In [34]:
# glmm correlation di = 5 
mean_empirical_cor_5_GLMM

0.7909588752324979

In [35]:
# glmm correlation di = 10
mean_empirical_cor_10_GLMM

0.7931953599572975

In [36]:
# glmm correlation di = 25
mean_empirical_cor_25_GLMM

0.7939346055300571